In [1]:
import numpy as np
import random
import torch
from src.dataloaders import Dataset, PairwiseDataset
from src.models import MatrixFactorizationRMSEModel, MatrixFactorizationBPRModel
from src.trainer import Trainer
from src.metrics import hitratio, ndcg
from sklearn.model_selection import GridSearchCV
from joblib import Parallel, delayed

np.random.seed(42)
random.seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f"{device=}")

device='cuda'


In [2]:
class config:
    data_dir = 'ml-100k'
    neg_count = 10
    epochs = 50
    batch_size = 2048
    dim = 40
    lr = 0.1

# dataset = Dataset(config.data_dir)
dataset = PairwiseDataset(config.data_dir)
dataset.gen_adjacency()
dataset.make_train_test()
print(f"{dataset.train_size=}, {dataset.test_size=}")

metrics = {
    # "HR@1": (hitratio, {"top_n": 1}),
    # "HR@5": (hitratio, {"top_n": 5}),
    "HR@10": (hitratio, {"top_n": 10}),
    # "NDCG@1": (ndcg, {"top_n": 1}),
    # "NDCG@5": (ndcg, {"top_n": 5}),
    "NDCG@10": (ndcg, {"top_n": 10}),
}

dataset.train_size=100000, dataset.test_size=943


In [7]:
# took 1 hour 15 mins to complete
grid_params = {
    "lr": [0.001, 0.01, 0.1, 1],
    "momentum": list(np.arange(0.1, 1.1, 0.1)),
    "weight_decay": [0.0001, 0.001, 0.01, 0.1, 1],
}

def search(lr, mom, wd):
    # model = MatrixFactorizationRMSEModel(dataset.user_count, dataset.item_count, config.dim)
    model = MatrixFactorizationBPRModel(dataset.user_count, dataset.item_count, config.dim)

    # optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=0.1)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=mom, nesterov=True, weight_decay=wd)

    trainer = Trainer(dataset, model, optimizer, metrics, 
                    epochs=config.epochs, batch_size=config.batch_size,
                    device=device, verbose=False, pbar=False)
    
    trainer.train(evaluate=True)
    best_ndcg, best_ndcg_epoch = float('-inf'), 0
    for i, s in enumerate(trainer.test_log):
        if s['NDCG@10'] > best_ndcg:
            best_ndcg = s['NDCG@10']
            best_ndcg_epoch = i

    return(best_ndcg, best_ndcg_epoch, (lr, mom, wd))

output = Parallel(n_jobs=4)(delayed(search)(lr, mom, wd) for lr in grid_params['lr'] for mom in grid_params['momentum'] for wd in grid_params['weight_decay'])

In [20]:
ind = np.argmax(np.array(output_generator)[:, 0])
ind, output_generator[ind]

/tmp/ipykernel_9181/4076919080.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ind = np.argmax(np.array(output_generator)[:, 0])


(127, (0.5835687783695144, 49, (0.1, 0.6, 0.01)))

In [23]:
sorted_log = sorted(output_generator)
sorted_log.reverse()
sorted_log

with open("grid_log", "w") as f:
    for r in sorted_log:
        f.write(f"{r}\n")

In [ ]:
grid_params = {
    "lr": [0.001, 0.01, 0.1, 1],
    "momentum": list(np.arange(0.1, 1.1, 0.1)),
    "weight_decay": [0.0001, 0.001, 0.01, 0.1, 1],
}

best = float('-inf')
best_params = None
for lr in grid_params['lr']:
    for mom in grid_params['momentum']:
        for wd in grid_params['weight_decay']:
            # model = MatrixFactorizationRMSEModel(dataset.user_count, dataset.item_count, config.dim)
            model = MatrixFactorizationBPRModel(dataset.user_count, dataset.item_count, config.dim)

            # optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=0.1)
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=mom, nesterov=True, weight_decay=wd)

            trainer = Trainer(dataset, model, optimizer, metrics, 
                            epochs=config.epochs, batch_size=config.batch_size,
                            device=device, verbose=False)
            
            trainer.train(evaluate=True)
            best_ndcg, best_ndcg_epoch = float('-inf'), 0
            for i, s in enumerate(trainer.test_log):
                if s['NDCG@10'] > best_ndcg:
                    best_ndcg = s['NDCG@10']
                    best_ndcg_epoch = i
            
            if best_ndcg > best:
                best = best_ndcg
                best_params = [lr, mom, wd, best_ndcg_epoch]

            print(best, best_params, (lr, mom, wd))

In [ ]:
# model = MatrixFactorizationRMSEModel(dataset.user_count, dataset.item_count, config.dim)
model = MatrixFactorizationBPRModel(dataset.user_count, dataset.item_count, config.dim)

# optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=0.1)
optimizer = torch.optim.SGD(model.parameters(), lr=config.lr, weight_decay=0.1)

trainer = Trainer(dataset, model, optimizer, metrics, 
                  epochs=config.epochs, batch_size=config.batch_size,
                  device=device, verbose=False)

In [ ]:
trainer.train(evaluate=True)

#### Epoch 49: Avg Loss/Batch 20.135686           
lr=0.1, weight_decay=0.01, batch_size=256                                               
                                                   
- HR@1: 0.27465535524920465
- HR@5: 0.6246023329798516
- HR@10: 0.7762460233297985
- NDCG@1: 0.27465535524920465
- NDCG@5: 0.45567622373046285
- NDCG@10: 0.5050744632837261

#### Epoch 49: Avg Loss/Batch 2.268884            
lr=0.1, weight_decay=0.001, batch_size=256                             

- HR@1: 0.271474019088017
- HR@5: 0.6521739130434783
- HR@10: 0.8154825026511134
- NDCG@1: 0.271474019088017
- NDCG@5: 0.46938699909136805
- NDCG@10: 0.5226068198556473

#### Epoch 49: Avg Loss/Batch 1.926479            
lr=0.1, weight_decay=0.001, batch_size=128                             

- HR@1: 0.31919406150583246
- HR@5: 0.7104984093319194
- HR@10: 0.8504772004241782
- NDCG@1: 0.31919406150583246
- NDCG@5: 0.5225957334522389
- NDCG@10: 0.5683831099482718

#### Epoch 49: Avg Loss/Batch 1.937064            
lr=0.1, weight_decay=0.001, batch_size=128                             
 
- HR@1: 0.3297985153764581
- HR@5: 0.6839872746553552
- HR@10: 0.848356309650053
- NDCG@1: 0.3297985153764581
- NDCG@5: 0.5171409534856304
- NDCG@10: 0.5709949698866551

#### Epoch 99: Avg Loss/Batch 1.799917            
lr=0.1, weight_decay=0.001, batch_size=128        
                                                   
- HR@1: 0.3244962884411453
- HR@5: 0.7232237539766702
- HR@10: 0.8642629904559915
- NDCG@1: 0.3244962884411453
- NDCG@5: 0.5335037960184896
- NDCG@10: 0.5796056642923276

#### Epoch 143: Avg Loss/Batch 1.769505            
lr=0.1, weight_decay=0.001, batch_size=128                                                   

- HR@1: 0.3372216330858961
- HR@5: 0.7253446447507953
- HR@10: 0.8685047720042418
- NDCG@1: 0.3372216330858961
- NDCG@5: 0.5399725224060331
- NDCG@10: 0.5865801610610061

#### Epoch 99: Avg Loss/Batch 342.869812          
lr=0.5, weight_decay=0.1, batch_size=131072                                                   

HR@10: 0.8430540827147401
NDCG@10: 0.554626786962534

In [ ]:
torch.save(trainer.model.state_dict(), "saved_models/mf.pt")
# trainer.model.load_state_dict(torch.load("saved_models/mfbpr.pt"))